In [1]:
import pandas as pd
data=pd.read_csv('datassss.csv')
data=data.sample(frac=1.0).reset_index(drop=True)

In [2]:
import tensorflow as tf
train_feature=data.iloc[:,1:-1]
train_label=data.iloc[:,-1:]

In [3]:
train_feature=tf.expand_dims(train_feature,-1)
train_feature.shape

TensorShape([2800, 29, 1])

In [4]:
class CNN_Decision(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1=tf.keras.layers.Conv1D(filters=10,kernel_size=200,padding='same')
        self.pool1=tf.keras.layers.MaxPool1D(pool_size=3,strides=2)
        self.reshape1=tf.keras.layers.Reshape(target_shape=(140,))
        self.dense1=tf.keras.layers.Dense(10,activation='relu')
        self.dense2=tf.keras.layers.Dense(1)
    def call(self, inputs):
        x=self.conv1(inputs)
        x=self.pool1(x)
        x=self.reshape1(x)
        x=self.dense1(x)
        x=self.dense2(x)
        output=x
        # output=tf.nn.softmax(x)
        return output

In [5]:
batch_size=100
rate=0.001
batch_num=int(len(train_feature)/batch_size)
epoch=10
model=CNN_Decision()
optimizer=tf.keras.optimizers.Adam(learning_rate=rate)
sunshi=[]
def getbatch(num):
    train=train_feature[100*num:100*(num+1)]
    test=train_label[100*num:100*(num+1)]
    return train,test
for i in range(epoch):
    for j in range(batch_num):
        temp_train,temp_test=getbatch(j)
        with tf.GradientTape() as tape:
            labels_pred=model(temp_train)
            #loss=tf.keras.losses.sparse_categorical_crossentropy(y_true=temp_test,y_pred=labels_pred)
            loss=tf.square(labels_pred-temp_test)
            loss=tf.reduce_mean(loss)
            sunshi.append((loss.numpy()))
        grads=tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads,model.trainable_variables))

In [6]:
import pandas as pd
predict=pd.read_csv('yuece.csv')
predict_feature=predict.iloc[:100,1:]
predict_feature=tf.expand_dims(predict_feature,-1)
predict_feature.shape

TensorShape([100, 29, 1])

In [24]:
predict=model(predict_feature)
predict=predict.numpy()
predict=predict.flatten()
predict=list(predict)

In [28]:
import numpy as np
predict=np.array(predict)
itemindex = np.argwhere( predict>1)
itemindex

array([[85]], dtype=int64)